In [1]:
import pyesapi
import atexit
import time
import os
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

In [2]:
import pandas as pd


def d_at_v(plan,structure,volume):
    _dose=plan.GetDoseAtVolume(structure,volume,
                               pyesapi.VolumePresentation.Relative,
                               pyesapi.DoseValuePresentation.Absolute)
    return _dose.Dose

columns = (
    'PatientId',
    'PlanId',
    'StructureId',
    'DicomType',
    'Volume(cc)',
    'IsHighRes',
    'D95%(Gy)',
    'D25%(Gy)',
    'D50%(Gy)'
)

def get_structure_and_dose_info(plan, get_volumes=True, get_dose=True):
    returning_list = []
    for s in plan.StructureSet.Structures:
        structure_list = [
        plan.StructureSet.Patient.Id,
        plan.Id,
        s.Id,
        s.DicomType,
        ]
        if get_volumes:
            structure_list += [
                s.Volume,
                s.IsHighResolution
            ]
        else:
            structure_list += [
                'NA',
                'NA'
            ]
        if get_dose:
            structure_list += [
                d_at_v(plan,s,95),
                d_at_v(plan,s,25),
                d_at_v(plan,s,50)]
        else:
            structure_list += [
                'NA', 'NA', 'NA'
            ]
        returning_list.append(tuple(structure_list))
    return returning_list

In [3]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
fid.close()
MRN = line.strip('\n')
app.ClosePatient()
patient = app.OpenPatientById(MRN)
plan = patient.CoursesLot(0).PlanSetupsLot(0)

In [4]:
plans = [i for i in patient.CoursesLot(0).PlanSetups]

In [5]:
[i.StructureSet for i in plans]

In [7]:
def returndataframe_list():
    dataframe_list = []
    patients_checked = []
    get_volumes = False
    global_get_dose = False
    i = 0
    for pat_sum in app.PatientSummaries:
        time.sleep(0.0)
        i += 1
        if i > 100:
            break
        MRN = pat_sum.Id
        print('MRN: {}'.format(MRN))
        if MRN in patients_checked:
            continue
        patients_checked.append(MRN)
        app.ClosePatient()
        patient = app.OpenPatientById(MRN)
        for course in patient.Courses:
            time.sleep(0.0)
            print('Course: {}'.format(course.Id))
            if course.Id.find('QA') != -1 or course.Id.find('use') != -1:
                continue
            print(plans)
            for plan in course.PlanSetups:
                time.sleep(0.0)
                print(i)
                print('Plan: {}'.format(plan.Id))
                if plan.StructureSet is None: # No structures present, move on
                    continue
                if global_get_dose and plan.Dose is not None:
                    get_dose = True
                else:
                    get_dose = False
                dataframe_list.append(pd.DataFrame(get_structure_and_dose_info(plan, get_volumes=get_volumes, get_dose=get_dose),
                                                   columns=columns))
    return dataframe_list
dataframe_list = returndataframe_list()

MRN: HEADNECK
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
1
Plan: Plan ID
MRN: 21002902
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
2
Plan: Plan1
MRN: 7878
MRN: 08136285
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x0

15
Plan: Plan1
15
Plan: Plan1:1
MRN: 19410703
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
16
Plan: 7 Final Rt e:1
16
Plan: 5 Lt.Neck e-
16
Plan: 3 OC  H and N
16
Plan: 4 Rt.Neck e-
16
Plan: 2 Neck:4
16
Plan: 6 Final Boost
16
Plan: 4 Rt.Neck e-:1
16
Plan: 5 Lt.Neck e-:1
16
Plan: 7 Final Rt e
16
Plan: Neck:3
16
Plan: 6 Final Boost:2
16
Plan: 6 Final Boost:1
16
Plan: 7 Final Rt e:2
16
Plan: 6 Final Boost:3
16
Plan: 6 Final Boost:4
16
Plan: 7 Final Rt e:3
16
Plan: Neck
16
Plan: 1 AP SCLAV
16
Plan: Neck:1
16
Plan: Neck:2
MRN: 20223038
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.External

33
Plan: Initial
33
Plan: offcord
33
Plan: Initial:2
33
Plan: Initial:1
33
Plan: Initial:3
MRN: 12969705
Course: Lt Nasal Bridge
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
34
Plan: bolus
34
Plan: bolus:1
34
Plan: bolus:2
MRN: 18479220
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
35
Plan: bolus
35
Plan: 1 bolus:1
35
Plan: no bolus:1
35
Plan: 1a no bolus:3
35
Plan: no bolus
35
Plan: 3a Boost
35
Plan: 1a no bolus:2
35
Plan: 3 Boost:1
35
Plan: 3 Boost:2
35
Plan: 3 Boost
MRN:

Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
84
Plan: 3Sclav:1
84
Plan: 3S/clav.'s
84
Plan: 1HN30EDW:2
84
Plan: 1aHN15W
84
Plan: 1H/N30EDW
84
Plan: 1HN30EDW:1
MRN: 18576025
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749850>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749640>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF7495B0>]
85
Plan: Plan2:1
85
Plan: Plan2
85
Plan: Plan1:2
85
Plan: Plan1
85
Plan: Plan1:1
MRN: 15784481
Course: C1
[<VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF749580>, <VMS.TPS.Common.Model.API.ExternalPlanSetup object at 0x000001BDFF

In [8]:
structure_dataframe = pd.concat(dataframe_list,ignore_index=True)
structure_dataframe

,PatientId,PlanId,StructureId,DicomType,Volume(cc),IsHighRes,D95%(Gy),D25%(Gy),D50%(Gy)
0,11076619,041217 RP_RML,BODY,EXTERNAL,NA,NA,NA,NA,NA
1,11076619,041217 RP_RML,carina,ORGAN,NA,NA,NA,NA,NA
2,11076619,041217 RP_RML,Cord,ORGAN,NA,NA,NA,NA,NA
3,11076619,041217 RP_RML,GTV,CTV,NA,NA,NA,NA,NA
4,11076619,041217 RP_RML,PTV,PTV,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...
295,18632232,2 T7-T11SPINE,carina,,NA,NA,NA,NA,NA
296,18632232,1 Lt Humerus,Dose 50[%] stern,CONTROL,NA,NA,NA,NA,NA
297,18632232,1 Lt Humerus,body,EXTERNAL,NA,NA,NA,NA,NA
298,18632232,1 Lt Humerus,gtv1,GTV,NA,NA,NA,NA,NA


In [14]:
structure_dataframe.to_excel(os.path.join('.', 'StructureData.xlsx'), index=False)

In [10]:
# if you insist
with open('./StructureData.csv','w') as f:
    f.write(structure_dataframe.to_csv(sep=','))

In [ ]:
i = 0
for pat_sum in app.PatientSummaries:
    print(i)
    i += 1
    if i > 200:
        break
    MRN = pat_sum.Id
    if MRN in patients_checked:
        continue
    patients_checked.append(MRN)
    app.ClosePatient()
    patient = app.OpenPatientById(MRN)
    courses = patient.Courses
    for course in courses:
        if course.Id.find('QA') == -1:
            plans = [i for i in course.PlanSetups]
            for plan in plans:
                if plan.StructureSet is None: # No structures present, move on
                    continue
                for structure in plan.StructureSet.Structures:
                    structure_name = structure.Id
                    if structure_name not in structures:
                        structures[structure_name] = {'PatientID': [], 'Course': [], 'Plan': []}
                    structures[structure_name]['PatientID'].append(MRN)
                    structures[structure_name]['Course'].append(course.Id)
                    structures[structure_name]['Plan'].append(plan.Id)

In [ ]:
structures['Lung RT']

In [ ]:
list(structures.keys())

In [ ]:
structures.keys()

In [ ]:
patient.CoursesLot('C1').PlanSetupsLot(0)